In [35]:
%matplotlib inline

from itertools import combinations
import os
import sys

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from scipy.stats import spearmanr

pd.set_option("display.max_columns", 101)
pd.set_option("display.float_format", lambda x: "%.2f" % x )

# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src', 'data')
sys.path.append(src_dir)

# import my method from the source code
%aimport utils

/Users/bull/anaconda/envs/farmdrive/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Geographic Features

Query raster data to create features for the counties.

We query the postgis database to get information about rainfall, elevation, soil properties and agro-ecological zones for each county in Kenya.

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('postgresql://localhost/farmdrive')
session = sessionmaker(bind=engine)()

In [15]:
def postgis_rater_in_geom(session, rast_table, geom_table, geom_group_id, feature_name=None):
    """ Queries a raster table for all the summary stats within the geoms
        in geom_table and groups results by geom_group_id.
        
        For example, query the trmm_acc_2015 rainfall raster table
        by county with postgis_rater_in_geom(session, 'trmm_acc_2015', 'county', 'county')
        
        Returns: A dataframe with the geom_group_id as the index and the summary
                 stats as the columns.
                 
        This method works for querying a raster with an existing shapefile. (To
        query a shapefile (polygon) instead, see postgis_geom_in_geom_summary).
    """
    
    # If unsupported characters in the table name, make
    # sure to quote it
    if "-" in rast_table:
        rast_table = '"{}"'.format(rast_table)
        
    if "-" in geom_table:
        geom_table = '"{}"'.format(geom_table)

    q_fmt = """
        SELECT {1}.{2}, (stats).*
         FROM {0} 
          INNER JOIN {1}
          ON ST_Intersects({0}.rast, {1}.geom), 
            ST_SummaryStats(ST_Clip({0}.rast, {1}.geom), 1) AS stats
         ORDER BY county.county;
    """
        
    # Execute the query in the session
    result = session.execute(q_fmt.format(rast_table, geom_table, geom_group_id))
    
    if feature_name is None:
        feature_name = rast_table
    
    # Put results into a dataframe
    df_columns = ['{}_{}'.format(geom_table, geom_group_id)] + \
                 ['{}_{}'.format(feature_name, c) for c in ['count', 'sum', 'mean', 'stddev', 'min', 'max']]
    query_df = pd.DataFrame(result.fetchall(),
                            columns=df_columns).set_index('county_county')
    
    # Return all the results
    return query_df

test_res = postgis_rater_in_geom(session, 'aez16_clas--ssa', 'county', 'county', 'agro_eco_zone')
test_res

,agro_eco_zone_count,agro_eco_zone_sum,agro_eco_zone_mean,agro_eco_zone_stddev,agro_eco_zone_min,agro_eco_zone_max
county_county,,,,,,
Baringo,122,1352.00,11.08,2.10,8.00,14.00
Bomet,32,448.00,14.00,0.00,14.00,14.00
Bungoma,33,462.00,14.00,0.00,14.00,14.00
Busia,21,262.00,12.48,1.94,10.00,14.00
Embu,35,368.00,10.51,1.98,9.00,14.00
Garissa,527,4020.00,7.63,0.48,7.00,8.00
Homa Bay,56,672.00,12.00,2.09,9.00,14.00
Isiolo,295,2418.00,8.20,0.84,8.00,13.00
Kajiado,256,2809.00,10.97,2.20,8.00,13.00


In [20]:
def raster_feature_by_county(session, rast_table, feature_name=None):
    """ Aggregates a raster table by county across multiple non-overlapping
        rasters if necessary. For example, our elevation data from SRTM
        has multiple non-overlapping tiles we need to query in order
        to cover every county in Kenya.
    """
    
    # If we need to query data from multiple raster tables, do so
    if isinstance(rast_table, list):
        pixel_dfs = [postgis_rater_in_geom(session, r, 'county', 'county', feature_name=feature_name) \
                         for r in rast_table]
        pixel_df = pd.concat(pixel_dfs).reset_index().groupby('county_county').mean()
    else:
        pixel_df = postgis_rater_in_geom(session, rast_table, 'county', 'county', feature_name=feature_name)
        
    return pixel_df

raster_feature_by_county(session, ['srtm_43_12', 'srtm_43_13', 'srtm_44_12', 'srtm_44_13', 'srtm_45_12', 'srtm_45_13'], feature_name='elevation')

,elevation_count,elevation_sum,elevation_mean,elevation_stddev,elevation_min,elevation_max
county_county,,,,,,
Baringo,631839.50,857796925.50,1779.94,365.70,1134.50,2954.00
Bomet,329808.00,650163655.00,1971.34,181.21,1581.00,2465.00
Bungoma,177009.00,328192383.00,1816.31,323.73,1436.00,3071.00
Busia,106286.00,129191250.50,1176.51,35.85,1116.50,1362.00
Embu,330699.00,395692483.00,1196.53,465.59,517.00,5035.00
Garissa,1312242.50,196121933.75,169.38,47.36,68.75,316.50
Homa Bay,278188.00,349679183.50,1402.38,87.51,1301.00,1954.00
Isiolo,1483262.00,677804899.00,398.83,148.42,225.00,1200.50
Kajiado,2554592.00,3288941077.00,1287.46,308.68,585.00,2546.00


In [21]:
# all raster features
table_feature_map = {
    "rainfall_2012": "trmm_acc_2012",
    "rainfall_2013": "trmm_acc_2013",
    "rainfall_2014": "trmm_acc_2014",
    "rainfall_2015": "trmm_acc_2015",
    "elevation": ['srtm_43_12', 'srtm_43_13', 'srtm_44_12', 'srtm_44_13', 'srtm_45_12', 'srtm_45_13'],
    "agro_ecological_zone": "aez16_clas--ssa",
}

all_features = []
for col_name, db_table in table_feature_map.items():
    f = raster_feature_by_county(session, db_table, feature_name=col_name)
    all_features.append(f)
    
county_raster_features = pd.concat(all_features, axis=1)

In [31]:
def postgis_geom_in_geom_summary(session,
                                 value_table,
                                 value_columns,
                                 geom_table,
                                 geom_group_id,
                                 feature_name=None):
    """ Querys `value_table` for summary statistics of each `value_column` aggregated
        by the geom_group_id in the geom_table.
        
        This method works for querying a shapefile with an existing shapefile. (To
        query a raster instead, see postgis_rater_in_geom).
    """

    q_fmt = """
        SELECT
          {1}.{2},

          {3}

        FROM
          {1},
          {0}
        WHERE
          ST_Contains({1}.geom, {0}.geom)
        GROUP BY {1}.{2};
    """
    
    summary_format = """
          AVG({0}.{1}) {1}_avg,
          MIN({0}.{1}) {1}_min,
          MAX({0}.{1}) {1}_max,
          stddev_samp({0}.{1}) {1}_std"""
    
    summaries = ',\n'.join([summary_format.format(value_table, c) for c in value_columns])
    
    q = q_fmt.format(value_table, geom_table, geom_group_id, summaries)
    
    # Execute the query in the session
    result = session.execute(q)
    
    if feature_name is None:
        feature_name = rast_table
    
    # Put results into a dataframe
    df_columns = ['{}_{}'.format(geom_table, geom_group_id)] + \
                 ['{}_{}'.format(f, c) for f in value_columns for c in ['avg', 'min', 'max', 'stddev']]
        
    query_df = pd.DataFrame(result.fetchall(),
                            columns=df_columns).set_index('county_county')
    
    # Return all the results
    return query_df

all_soil_params = [
    'cfrag',
    'sdto',
    'stpc',
    'clpc',
    'bulk',
    'tawc',
    'cecs',
    'bsat',
    'cecc',
    'phaq',
    'tceq',
    'gyps',
    'elco',
    'totc',
    'totn',
    'ecec'
]

all_soil_summary = postgis_geom_in_geom_summary(session,
                                                'soilparameterestimated_t1s1d1',
                                                all_soil_params,
                                                'county',
                                                'county',
                                                feature_name='soil')
all_soil_summary

,cfrag_avg,cfrag_min,cfrag_max,cfrag_stddev,sdto_avg,sdto_min,sdto_max,sdto_stddev,stpc_avg,stpc_min,stpc_max,stpc_stddev,clpc_avg,clpc_min,clpc_max,clpc_stddev,bulk_avg,bulk_min,bulk_max,bulk_stddev,tawc_avg,tawc_min,tawc_max,tawc_stddev,cecs_avg,cecs_min,cecs_max,cecs_stddev,bsat_avg,bsat_min,bsat_max,bsat_stddev,cecc_avg,cecc_min,cecc_max,cecc_stddev,phaq_avg,phaq_min,phaq_max,phaq_stddev,tceq_avg,tceq_min,tceq_max,tceq_stddev,gyps_avg,gyps_min,gyps_max,gyps_stddev,elco_avg,elco_min,elco_max,elco_stddev,totc_avg,totc_min,totc_max,totc_stddev,totn_avg,totn_min,totn_max,totn_stddev,ecec_avg,ecec_min,ecec_max,ecec_stddev
county_county,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Baringo,11.1428571428571429,0.00000,60.00000,15.8646276713267300,42.0952380952380952,10.00000,77.00000,22.7551222041828166,29.6190476190476190,6.00000,60.00000,14.2314532808663641,28.2857142857142857,10.00000,65.00000,14.1443528579442207,1.3095238095238095,1.00000,1.61000,0.11910135837017716609,13.1666666666666667,8.00000,27.00000,4.2593751211608761,19.2919047619047619,10.00000,56.20000,9.0053951953734279,87.5000000000000000,23.00000,100.00000,21.7426746845696146,54.6666666666666667,15.00000,140.00000,19.3978458279322254,7.0547619047619048,4.50000,8.88000,1.0994416278945885,33.4995240476190476,0.00000,307.50000,72.2996676757342030,0E-20,0.00000,0.00000,0,0.01285714285714285714,0.00000,0.40000,0.06489337274162880286,10.9592857142857143,1.35000,38.20000,8.5564200380530536,1.0530952380952381,0.29000,5.10000,0.73874807777468740195,-0.67857142857142857143,-1.00000,12.50000,2.0830952244882408
Isiolo,2.6527777777777778,0.00000,28.00000,6.7410028138131826,45.5555555555555556,15.00000,92.00000,23.6261307286685798,17.1805555555555556,2.00000,36.00000,7.3068102304185362,37.2638888888888889,6.00000,63.00000,20.5007681446986690,1.3777777777777778,1.10000,1.68000,0.14666453263492283154,10.6805555555555556,2.00000,16.00000,3.6572381979174845,18.1631944444444444,3.80000,40.00000,8.7707501769166926,79.6944444444444444,23.00000,100.00000,26.9494251886171054,45.7916666666666667,11.00000,133.00000,22.1981711592847533,6.8697222222222222,5.00000,8.87000,1.3227212370898538,14.0877779166666667,0.00000,163.32001,27.4053841294542611,0E-20,0.00000,0.00000,0,0E-20,0.00000,0.00000,0,9.2554166666666667,1.55000,19.00000,5.3811067340370226,1.0681944444444444,0.28000,2.07000,0.51455252373981312990,-0.35958333333333333333,-1.00000,18.00000,3.2679460911908534
Tana River,0.25954198473282442748,0.00000,1.00000,0.44006618837034022367,40.6106870229007634,10.00000,83.00000,22.0159286808032451,18.4656488549618321,3.00000,35.00000,9.6103291466274926,40.9236641221374046,11.00000,60.00000,17.4499805248340461,1.4454198473282443,1.20000,1.63000,0.13083318528677731597,11.6335877862595420,7.00000,18.00000,3.8214095403301908,20.2082442748091603,4.20000,50.00000,11.9073528984221814,92.7786259541984733,45.00000,100.00000,12.8340233372295208,42.3740458015267176,-1.00000,140.00000,26.5817565773285122,7.3661832061068702,5.00000,9.00000,1.1788608960904507,26.2633587786259542,0.00000,187.05000,49.4316886157154959,0E-20,0.00000,0.00000,0,0.01526717557251908397,0.00000,2.00000,0.17474081133220758839,8.4596183206106870,2.00000,23.50000,5.4318728041894117,0.91870229007633587786,0.46000,2.98000,0.39778501527751217324,-0.10099236641221374046,-1.00000,8.52000,2.6157925273414033
Marsabit,8.2142857142857143,0.00000,60.00000,13.6029069508842057,47.7946428571428571,10.00000,98.00000,18.7385305514102971,21.3526785714285714,2.00000,50.00000,10.0637293038714797,30.8526785714285714,0.00000,60.00000,14.9207380275358665,1.3261607142857143,0.82000,1.70000,0.15300426756681629107,12.2633928571428571,3.00000,27.00000,4.9741303052294142,21.0454464285714286,3.05000,88.40000,11.6744143524398497,88.6562500000000000,23.00000,100.00000,24.9726784563152378,59.4419642857142857,-1.00000,140.00000,32.7345106311184749,7.5550892857142857,5.00000,11.00000,1.1311968308032066,49.0666522321428571,0.00000,307.50000,69.0274233

In [33]:
county_geo_features = county_raster_features.join(all_soil_summary)
county_geo_features

,rainfall_2014_count,rainfall_2014_sum,rainfall_2014_mean,rainfall_2014_stddev,rainfall_2014_min,rainfall_2014_max,rainfall_2012_count,rainfall_2012_sum,rainfall_2012_mean,rainfall_2012_stddev,rainfall_2012_min,rainfall_2012_max,agro_ecological_zone_count,agro_ecological_zone_sum,agro_ecological_zone_mean,agro_ecological_zone_stddev,agro_ecological_zone_min,agro_ecological_zone_max,rainfall_2013_count,rainfall_2013_sum,rainfall_2013_mean,rainfall_2013_stddev,rainfall_2013_min,rainfall_2013_max,rainfall_2015_count,rainfall_2015_sum,rainfall_2015_mean,rainfall_2015_stddev,rainfall_2015_min,rainfall_2015_max,elevation_count,elevation_sum,elevation_mean,elevation_stddev,elevation_min,elevation_max,cfrag_avg,cfrag_min,cfrag_max,cfrag_stddev,sdto_avg,sdto_min,sdto_max,sdto_stddev,stpc_avg,stpc_min,stpc_max,stpc_stddev,clpc_avg,clpc_min,clpc_max,clpc_stddev,bulk_avg,bulk_min,bulk_max,bulk_stddev,tawc_avg,tawc_min,tawc_max,tawc_stddev,cecs_avg,cecs_min,cecs_max,cecs_stddev,bsat_avg,bsat_min,bsat_max,bsat_stddev,cecc_avg,cecc_min,cecc_max,cecc_stddev,phaq_avg,phaq_min,phaq_max,phaq_stddev,tceq_avg,tceq_min,tceq_max,tceq_stddev,gyps_avg,gyps_min,gyps_max,gyps_stddev,elco_avg,elco_min,elco_max,elco_stddev,totc_avg,totc_min,totc_max,totc_stddev,totn_avg,totn_min,totn_max,totn_stddev,ecec_avg,ecec_min,ecec_max,ecec_stddev
county_county,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Baringo,15,185.42,12.36,9.20,0.62,27.71,15,36.78,2.45,2.84,0.00,10.96,122,1352.00,11.08,2.10,8.00,14.00,15,531.58,35.44,8.41,19.41,50.47,15,119.79,7.99,16.95,0.00,70.67,631839.50,857796925.50,1779.94,365.70,1134.50,2954.00,11.1428571428571429,0.00000,60.00000,15.8646276713267300,42.0952380952380952,10.00000,77.00000,22.7551222041828166,29.6190476190476190,6.00000,60.00000,14.2314532808663641,28.2857142857142857,10.00000,65.00000,14.1443528579442207,1.3095238095238095,1.00000,1.61000,0.11910135837017716609,13.1666666666666667,8.00000,27.00000,4.2593751211608761,19.2919047619047619,10.00000,56.20000,9.0053951953734279,87.5000000000000000,23.00000,100.00000,21.7426746845696146,54.6666666666666667,15.00000,140.00000,19.3978458279322254,7.0547619047619048,4.50000,8.88000,1.0994416278945885,33.4995240476190476,0.00000,307.50000,72.2996676757342030,0E-20,0.00000,0.00000,0,0.01285714285714285714,0.00000,0.40000,0.06489337274162880286,10.9592857142857143,1.35000,38.20000,8.5564200380530536,1.0530952380952381,0.29000,5.10000,0.73874807777468740195,-0.67857142857142857143,-1.00000,12.50000,2.0830952244882408
Bomet,4,245.92,61.48,7.20,49.32,67.66,4,95.64,23.91,3.45,19.31,27.49,32,448.00,14.00,0.00,14.00,14.00,4,295.51,73.88,6.63,65.69,84.18,4,137.46,34.36,5.50,27.14,42.32,329808.00,650163655.00,1971.34,181.21,1581.00,2465.00,0.46153846153846153846,0.00000,1.00000,0.51887452166277082560,27.3076923076923077,9.00000,41.00000,15.2336503361506419,31.6923076923076923,24.00000,70.00000,12.2023537031769366,41.0000000000000000,15.00000,67.00000,18.9428966457966331,1.0946153846153846,0.93000,1.30000,0.12855708100654384771,13.9230769230769231,8.00000,17.00000,3.3030677271676417,21.4892307692307692,19.33000,31.60000,4.1849879759653264,41.3846153846153846,24.00000,80.00000,18.9144497740856912,37.5384615384615385,16.00000,111.00000,32.0848234752179591,5.2030769230769231,4.70000,6.50000,0.40941797337571407978,0.23076923076923076923,0.00000,3.00000,0.83205029433784368303,0E-20,0.00000,0.00000,0,0E-20,0.00000,0.00000,0,25.5476923076923077,5.00000,40.00000,8.7048972364661432,2.6038461538461538,0.47000,3.00000,0.72147462950934111529,0.24000000000000000000,-1.00000,15.12000,4.4708835815753467
Bungoma,2,75.93,37.97,11.74,26.22,49.71,2,6.96,3.48,0.11,3.37,3.60,33,462.00,14.00,0.00,14.00,14.00,2,96.31,48.16,5.79,42.37,53.95,2,12.68,6.34,0.83,5.51,7.17,177009.00,328192383.00,1816.31,323.73,1436.00,3071.00,8.2941176470588235,0.00000,52.00000,17.7755615448653019,50.7058823529411765,3.00000,86.00000,24.7859958088291082,23.4705882352941176,6.00000,3

In [34]:
# Rename counties to match the ones that we use in our targets.
targets = pd.read_csv("../data/processed/targets-by-county.csv", index_col=0)
county_geo_features.index = utils.resolve_county_names(county_geo_features.index,
                                                       targets.index)

county_geo_features.to_csv("../data/processed/county-geo-features.csv")